In [1]:
import pandas as pd
import sklearn as sk
from pathlib import Path
from sklearn.utils import resample

In [2]:
ctrain = pd.read_csv('./project_data/cases_2021_train_processed_2.csv')
ctest = pd.read_csv('./project_data/cases_2021_test_processed_unlabelled_2.csv')

In [3]:
ctrain

,age,sex,province,country,latitude,longitude,date_confirmation,chronic_disease_binary,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio,outcome_group
0,18,female,Bihar,India,25.49096,85.93903,2020-05-18,False,265527,1576,262371,1580,212.762145,0.593537,hospitalized
1,27,female,NaN,Philippines,7.07000,125.60000,2020-04-15,False,747288,13297,603746,130245,681.949809,1.779368,nonhospitalized
2,46,male,Tamil Nadu,India,13.08362,80.28252,2020-05-02,False,886673,12719,858075,15879,1139.078325,1.434463,hospitalized
3,21,female,Tamil Nadu,India,13.08362,80.28252,2020-05-24,False,886673,12719,858075,15879,1139.078325,1.434463,hospitalized
4,27,male,Bihar,India,26.28361,87.20347,2020-05-27,False,265527,1576,262371,1580,212.762145,0.593537,hospitalized
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17207,29,female,Tamil Nadu,India,13.08362,80.28252,2020-04-24,False,886673,12719,858075,15879,1139.078325,1.434463,hospitalized
17208,47,female,NaN,Philippines,16.68814,121.54992,2020-04-09,False,747288,13297,603746,130245,681.949809,1.779368,nonhospitalized
17209,30,male,Bihar,India,24.94452,86.26404,2020-05-26,False,265527,1576,262371,1580,212.762145,0.593537,hospitalized
17210,59,male,Tamil Nadu,India,12.68224,79.98008,2020-05-31,False,886673,12719,858075,15879,1139.078325,1.434463,hospitalized


In [4]:
ctrain

,age,sex,province,country,latitude,longitude,date_confirmation,chronic_disease_binary,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio,outcome_group
0,18,female,Bihar,India,25.49096,85.93903,2020-05-18,False,265527,1576,262371,1580,212.762145,0.593537,hospitalized
1,27,female,NaN,Philippines,7.07000,125.60000,2020-04-15,False,747288,13297,603746,130245,681.949809,1.779368,nonhospitalized
2,46,male,Tamil Nadu,India,13.08362,80.28252,2020-05-02,False,886673,12719,858075,15879,1139.078325,1.434463,hospitalized
3,21,female,Tamil Nadu,India,13.08362,80.28252,2020-05-24,False,886673,12719,858075,15879,1139.078325,1.434463,hospitalized
4,27,male,Bihar,India,26.28361,87.20347,2020-05-27,False,265527,1576,262371,1580,212.762145,0.593537,hospitalized
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17207,29,female,Tamil Nadu,India,13.08362,80.28252,2020-04-24,False,886673,12719,858075,15879,1139.078325,1.434463,hospitalized
17208,47,female,NaN,Philippines,16.68814,121.54992,2020-04-09,False,747288,13297,603746,130245,681.949809,1.779368,nonhospitalized
17209,30,male,Bihar,India,24.94452,86.26404,2020-05-26,False,265527,1576,262371,1580,212.762145,0.593537,hospitalized
17210,59,male,Tamil Nadu,India,12.68224,79.98008,2020-05-31,False,886673,12719,858075,15879,1139.078325,1.434463,hospitalized


In [5]:
ctrain.drop(columns=['province', 'country', 'date_confirmation'], inplace=True)
ctest.drop(columns=['province', 'country', 'date_confirmation'], inplace=True)

In [6]:
ctrain.outcome_group = ctrain.outcome_group.astype('category').cat.codes

ctrain.sex = ctrain.sex.astype('category').cat.codes
ctest.sex = ctest.sex.astype('category').cat.codes

ctrain.chronic_disease_binary = ctrain.chronic_disease_binary.astype('category').cat.codes
ctest.chronic_disease_binary = ctest.chronic_disease_binary.astype('category').cat.codes

In [7]:
ctrain.outcome_group.value_counts()

1    13241
2     2974
0      997
Name: outcome_group, dtype: int64

In [8]:
ctrain_minority = ctrain[ctrain.outcome_group==0]
ctrain_rest = ctrain[ctrain.outcome_group!=0]

ctrain_minority_dups = resample(ctrain_minority, 
                                replace=True,
                                n_samples=int(ctrain_rest.shape[0]/2))

ctrain = pd.concat([ctrain_rest, ctrain_minority_dups])

In [9]:
# TODO: Consider running this to upscale outcome group 2 - it is a minority class, but to a lesser extent than 0

# ctrain_minority = ctrain[ctrain.outcome_group==2]
# ctrain_rest = ctrain[ctrain.outcome_group!=2]

# ctrain_minority_dups = resample(ctrain_minority, 
#                                 replace=True,
#                                 n_samples=int(ctrain_rest.shape[0]/2))

# ctrain = pd.concat([ctrain_rest, ctrain_minority_dups])

In [10]:
ctrain.outcome_group.value_counts()

1    13241
0     8107
2     2974
Name: outcome_group, dtype: int64

In [11]:
# Shuffle training set
ctrain = ctrain.sample(frac=1)
ctrain = ctrain.reset_index(drop='True')
ctrain

,age,sex,latitude,longitude,chronic_disease_binary,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio,outcome_group
0,34,0,12.68224,79.98008,0,886673,12719,858075,15879,1139.078325,1.434463,1
1,52,1,-26.56750,31.49822,0,17337,667,16528,142,1494.357694,3.847263,2
2,79,1,11.66552,78.15164,0,886673,12719,858075,15879,1139.078325,1.434463,1
3,86,0,14.59580,120.97720,0,747288,13297,603746,130245,681.949809,1.779368,0
4,67,0,10.33333,123.75000,0,747288,13297,603746,130245,681.949809,1.779368,2
...,...,...,...,...,...,...,...,...,...,...,...,...
24317,27,0,13.08362,80.28252,0,886673,12719,858075,15879,1139.078325,1.434463,1
24318,97,0,14.63000,121.03000,0,747288,13297,603746,130245,681.949809,1.779368,0
24319,64,1,13.08362,80.28252,0,886673,12719,858075,15879,1139.078325,1.434463,0
24320,49,0,6.92130,122.07453,0,747288,13297,603746,130245,681.949809,1.779368,2


In [12]:
# Select X and y from training set
ctrain_y = ctrain['outcome_group']
ctrain_X = ctrain.loc[:, ctrain.columns != 'outcome_group']

In [13]:
# Partition database into test set and training set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ctrain_X, ctrain_y, test_size=0.2)

In [19]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X_train, y_train):
    X_train_cv, y_train_cv = X_train.iloc[train_index], y_train.iloc[train_index]
    X_val_cv, y_val_cv = X_train.iloc[test_index], y_train.iloc[test_index]
    print(X_val_cv)
    print(y_val_cv)

       age  sex   latitude   longitude  chronic_disease_binary  Confirmed  \
9191    40    1  25.213430   84.985350                       0     265527   
828     30    1  13.083620   80.282520                       0     886673   
24305   25    1  12.867673   75.251507                       0     997004   
13966   29    1  14.630000  121.030000                       0     747288   
8174    12    1  11.139270   79.084280                       0     886673   
...    ...  ...        ...         ...                     ...        ...   
5166    48    0  13.083620   80.282520                       0     886673   
19828   19    0  13.083620   80.282520                       0     886673   
14351   30    1   9.589200   77.953700                       0     886673   
10997   32    1  11.735770   78.963060                       0     886673   
24215   42    0  14.595800  120.977200                       0     747288   

       Deaths  Recovered  Active  Incident_Rate  Case_Fatality_Ratio  
9191